#0.初期設定

In [64]:
#*******今回の実行の設定*******

#プロジェクトのフォルダ　※BASE_DIRを、自らの環境に合わせて修正してください
BASE_DIR = '/content/drive/MyDrive/project01'

#モデル名
MODEL_NAME = "data1_trial1"

#動作モード：1:データ①、2:データ②、3:データ①コンペ時設定
ENV_MODE = 1

#遺伝的アルゴリズムの集団数、実行世代数
NUM_POPULATIONS = 30
NUM_GENERATIONS = 50

#乱数シード
RANDOM_SEED = 42



In [50]:
#*******Googleドライブへの接続*******
from google.colab import drive
drive.mount('/content/drive')

#*******不足ライブラリのインストール*******
!pip install sktime

#*******ライブラリのインポート*******
import sys
import os
import random
import pandas as pd
import numpy as np
import glob
import torch
from scipy.io import loadmat

#*******乱数シードを固定*******
seed = RANDOM_SEED
torch.manual_seed(seed)  # PyTorchのシードを固定
np.random.seed(seed)     # NumPyのシードを固定
random.seed(seed)        # Pythonのrandomモジュールのシードを固定
#CUDAにおけるシードの固定
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # マルチGPUを使用している場合
#CuDNNの再現性確保
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#*******定数設定*******
DATA_DIR1 = os.path.join(BASE_DIR, 'data1/modeling_data')
DATA_DIR2 = os.path.join(BASE_DIR, 'data2/modeling_data')

#被験者
SUBJECTS=['subject0','subject1','subject2','subject3','subject4']
#試行
TRAIN_MATS=['train1.mat','train2.mat','train3.mat']
#クラス
CLASSES=['backside_kickturn','frontside_kickturn','pumping']
#データ属性
NUM_SENSORS = 72
SEQ_LENGTH = 250
BATCH_SIZE = 32


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1.データローダ定義

In [36]:
import os
import math
import pandas as pd
import numpy as np
import random
import glob
import pickle
from torch.utils.data import Dataset, DataLoader

class SeqDataset(Dataset):
    def __init__(self, root, subjects, mats, mask, seq_length, is_train, transform=None, cache_file='./data_cache.pt', use_cache=True):

        self.transform = transform
        self.raw_seqs = [] #マスクで取捨選択するため、読み込んだ値を保存

        self.seqs = [] #get_itemで返す値
        self.seq_labels = []
        self.subjects = os.listdir(root)
        self.class_names = CLASSES
        self.class_names.sort()
        self.numof_classes = len(self.class_names)
        self.mask = mask
        self.seq_length = seq_length
        self.is_train = is_train
        self.cache_file = cache_file

        if os.path.exists(self.cache_file) and use_cache:
            self.load_data_from_cache()
        else:
            for (j, y) in enumerate(subjects):
                for mat in mats:
                    for (i, x) in enumerate(self.class_names):
                        temp = glob.glob(os.path.join(root, y, mat, x, '*'))
                        temp.sort()
                        self.seq_labels.extend([i] * len(temp))

                        for t in temp:
                            df = pd.read_csv(t, header=None)
                            tensor = self.preprocess(df)
                            self.raw_seqs.append(tensor)

            # ロードしたデータをキャッシュファイルに保存
            if self.cache_file!="":
                self.save_data_to_cache()

        self.seqs=self.blend_data(self.raw_seqs)

    def __getitem__(self, index):
        seq = self.seqs[index]
        if self.transform is not None:
            seq = self.transform(seq, is_train=self.is_train, seq_length=self.seq_length)

        #(タイムステップ,センサー)から(センサー,タイムステップ)に変換
        #seq = seq.T.astype(np.float32)

        return seq, self.seq_labels[index]

    def __len__(self):
        return len(self.seqs)


    #*******マスク処理関連*******
    def set_mask(self, mask):
        #マスクをセット
        self.mask = mask
        self.blend_data(self.raw_seqs)

    def blend_data(self, data1):
        #マスクに応じたデータセットマスキング処理
        seqs=[]
        for i in range(0, len(self.raw_seqs)):
            data1 = self.raw_seqs[i]

            result = np.zeros_like(data1)
            for j in range(NUM_SENSORS):
                if self.mask[j] == 1:
                    result[j, :] = data1[j, :]
                else:
                    pass #0のまま

            seqs.append(result)

        return seqs

    #*******データセットロード時の加工処理*******
    #※試行錯誤の結果、今回は標準化のみ
    def preprocess(self, df: pd.DataFrame) -> np.ndarray:
        mat = df.T.values

        #動作モード：1:データ①、2:データ②、3:データ①コンペ時設定
        if ENV_MODE!=1:  #データ1は、別途「car→標準化」を行うので標準化をパス。コンペ時はstd→car→stdを実施
            mat = self.standardization(mat, axis=1)

        return mat

    #各種前処理用関数
    def standardization(self, a, axis=None, ddof=0):
        a_mean = a.mean(axis=axis, keepdims=True)
        a_std = a.std(axis=axis, keepdims=True, ddof=ddof)
        a_std[np.where(a_std == 0)] = 1
        return (a - a_mean) / a_std

    def min_max_scaling(self, a, axis=None):
        min_val = np.min(a, axis=axis, keepdims=True)
        max_val = np.max(a, axis=axis, keepdims=True)
        return (a - min_val) / (max_val - min_val + 1e-8)

    #*******ロード高速化キャッシュ関連*******
    def save_data_to_cache(self):
        #キャッシュの保存
        os.makedirs(os.path.dirname(self.cache_file), exist_ok=True)
        with open(self.cache_file, 'wb') as f:
            pickle.dump((self.raw_seqs, self.seq_labels), f)
        print(f"キャッシュ生成：{self.cache_file}")

    def load_data_from_cache(self):
        #キャッシュの読込み
        with open(self.cache_file, 'rb') as f:
            self.raw_seqs, self.seq_labels = pickle.load(f)

#*******get_item時のデータ変換関数*******
def transform(array, is_train, seq_length):
    if is_train:
        #今回は何もしない　※Rocketにて、どれも効果が薄かったため
        ts = array[:, :seq_length].astype(np.float32)
        return ts
    else:
        ts = array[:, :seq_length].astype(np.float32)
        return ts


In [37]:
#*******動作確認*******
#mask=[0]*72
mask=[1]*72

fold=2 #ベースラインは2とする
subject=SUBJECTS[1]
train_mats=TRAIN_MATS.copy()
val_mats=[train_mats.pop(fold)]


train_dataset = SeqDataset(DATA_DIR1, [subject], train_mats, mask=mask,
                           seq_length=SEQ_LENGTH, is_train=True, transform=transform,
                           cache_file=os.path.join(DATA_DIR1, 'pkl', f'ds_train_{subject}_{fold}.pt'),
                           use_cache=True)
val_dataset = SeqDataset(DATA_DIR1, [subject], val_mats, mask=mask,
                           seq_length=SEQ_LENGTH, is_train=True, transform=transform,
                           cache_file=os.path.join(DATA_DIR1, 'pkl', f'ds_val_{subject}_{fold}.pt'),
                           use_cache=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
one_batch = next(iter(train_loader))

# ミニバッチの内容を確認
#print(one_batch)


#2.学習実行

In [ ]:
import os
import random
import time
import joblib
import glob
import json
from datetime import datetime
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import accuracy_score
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#*******上位モデルの保存*******
def save_mask_models(new_models, model_dir, mask, generation):
    #マスクモデルの保存管理

    mask_dir = os.path.join(model_dir, "mask_models")
    os.makedirs(mask_dir, exist_ok=True)

    # 新しいマスクの平均スコアを計算
    current_score = np.mean([score for _, _, score, _ in new_models])

    # 新しいマスクのディレクトリ名
    new_dir_name = f"score{current_score:.4f}_gen{generation+1}"
    new_dir_path = os.path.join(mask_dir, new_dir_name)

    # 既存のマスクディレクトリを取得してスコアとジェネレーション情報を抽出
    existing_dirs = []
    for dir_path in glob.glob(os.path.join(mask_dir, "score*_gen*")):
        dir_name = os.path.basename(dir_path)
        score = float(dir_name.split('_')[0].replace('score', ''))
        gen = int(dir_name.split('_')[1].replace('gen', ''))
        existing_dirs.append({
            'path': dir_path,
            'score': score,
            'generation': gen
        })

    # スコアで降順ソート（同点の場合は世代が新しい方を優先）
    existing_dirs.sort(key=lambda x: (x['score'], x['generation']), reverse=True)

    # 新しいスコアが上位3位以内に入るか確認
    should_save = False
    if len(existing_dirs) < 3:
        should_save = True
    else:
        min_top3_score = existing_dirs[2]['score']
        if current_score >= min_top3_score:
            should_save = True

    if should_save:
        # 新しいディレクトリを作成して全モデルを保存
        os.makedirs(new_dir_path, exist_ok=True)
        for subject, fold, _, pipeline in new_models:
            file_name = f"{subject}_{fold}.pkl"
            file_path = os.path.join(new_dir_path, file_name)
            joblib.dump(pipeline, file_path)

        # マスクを保存する
        with open(os.path.join(new_dir_path, 'mask.txt'), 'w', encoding='utf-8') as file:
            file.write(f'{mask}')

        # 既存のディレクトリリストに新しいディレクトリを追加
        existing_dirs.append({
            'path': new_dir_path,
            'score': current_score,
            'generation': generation
        })

        # 再度ソート
        existing_dirs.sort(key=lambda x: (x['score'], x['generation']), reverse=True)

        # 上位3つ以外のディレクトリを削除
        for dir_info in existing_dirs[3:]:
            import shutil
            try:
                shutil.rmtree(dir_info['path'])
            except Exception as e:
                print(f"モデル移動処理失敗：{dir_info['path']}: {str(e)}")

def update_model_rankings(new_models, model_dir, mask, generation):
    #モデルの保存を管理する関数

    try:
        # マスクモデルの保存
        save_mask_models(new_models, model_dir, mask, generation)

        return True
    except Exception as e:
        print(f"モデル更新エラー:{str(e)}")
        return False


#*******フィルタ適用処理*******
def apply_car_and_normalize(data):
    #Common Average Referenceと正規化を実施

    #CAR
    car_data = data - np.mean(data, axis=1, keepdims=True)

    #正規化
    mean = np.mean(car_data, axis=2, keepdims=True)
    std = np.std(car_data, axis=2, keepdims=True)
    normalized_data = (car_data - mean) / (std + 1e-8)  #1e-8：0除算回避

    return normalized_data

def prepare_data(train_loader, val_loader):
    #機械学習用データ作成
    #※DLから移行した名残り。ローダから全データを取り込んでテーブル化する

    X_train, y_train = [], []
    X_val, y_val = [], []

    for batch in train_loader:
        X, y = batch
        X_train.extend(X.numpy())
        y_train.extend(y.numpy())

    for batch in val_loader:
        X, y = batch
        X_val.extend(X.numpy())
        y_val.extend(y.numpy())

    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_val = np.array(X_val)
    y_val = np.array(y_val)

    # CAR と正規化を適用
    if ENV_MODE!=2:
        #動作モードにより実施有無変更：1:データ①、2:データ②、3:データ①コンペ時設定
        #※data2はクレンジング済みなので、CARは逆効果
        X_train = apply_car_and_normalize(X_train)
        X_val = apply_car_and_normalize(X_val)

    return X_train, y_train, X_val, y_val

#*******モデル学習・評価*******
# ROCKETモデルの学習関数
def train_rocket_model(X_train, y_train):
    rocket = MiniRocketMultivariate(num_kernels=10000, random_state=RANDOM_SEED)
    scaler = StandardScaler(with_mean=True)
    classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
    pipeline = make_pipeline(rocket, scaler, classifier)
    pipeline.fit(X_train, y_train)
    return pipeline

def evaluate_model(model_name, mask, generation):

    #動作モードによりデータ参照先変更：1:データ①、2:データ②、3:データ①コンペ時設定
    if ENV_MODE!=2:
        data_dir = DATA_DIR1
    else:
        data_dir = DATA_DIR2

    #保存先作成
    model_dir = os.path.join(BASE_DIR, f"models_{model_name}")
    os.makedirs(model_dir, exist_ok=True)

    new_models = []
    total_scores = []

    for subject in SUBJECTS:
        subject_scores = []
        for fold in range(3):

            train_mats=TRAIN_MATS.copy()
            val_mats=[train_mats.pop(fold)]

            train_dataset = SeqDataset(data_dir, [subject], train_mats, mask=mask,
                                      seq_length=SEQ_LENGTH, is_train=True, transform=transform,
                                      cache_file=os.path.join(data_dir, 'pkl', f'ds_train_{subject}_{fold}.pt'),
                                      use_cache=True)
            val_dataset = SeqDataset(data_dir, [subject], val_mats, mask=mask,
                                      seq_length=SEQ_LENGTH, is_train=False, transform=transform,
                                      cache_file=os.path.join(data_dir, 'pkl', f'ds_valid_{subject}_{fold}.pt'),
                                      use_cache=True)

            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

            X_train, y_train, X_val, y_val = prepare_data(train_loader, val_loader)
            pipeline = train_rocket_model(X_train, y_train)

            y_pred = pipeline.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)

            subject_scores.append(accuracy)
            new_models.append((subject, fold, accuracy, pipeline))

        total_scores.append(np.mean(subject_scores))

    # generationパラメータを追加して更新関数を呼び出し
    update_model_rankings(new_models, model_dir, mask, generation)

    avg_score = np.mean(total_scores)
    return avg_score


#*******遺伝的アルゴリズム関連*******
def initialize_population(population_size, mask_size):
    #初期の集団を生成する
    population = []
    # ランダムなマスク
    for _ in range(population_size - 2):
        mask = [random.choice([0, 1]) for _ in range(mask_size)]
        population.append(mask)
    # すべて0のマスク
    population.append([0] * mask_size)
    # すべて1のマスク
    population.append([1] * mask_size)
    return population

def crossover(parent1, parent2):
    #交差処理
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(mask, mutation_rate):
    #突然変異処理
    return [1 - bit if random.random() < mutation_rate else bit for bit in mask]

def adjust_mask(mask):
    #不使用：マスクの1が一定数を超えないようにする
    #※必要最低限のセンサーを探索するために使用した。将来用に残す

    SENSORS_LIMIT=999 #不使用のため、72以上にして、ヒットしないようにする

    ones_count = sum(mask)
    if ones_count > SENSORS_LIMIT:
        indices = [i for i, bit in enumerate(mask) if bit == 1]
        to_zero = random.sample(indices, ones_count - SENSORS_LIMIT)
        for i in to_zero:
            mask[i] = 0
    return mask

def genetic_algorithm(model_name, population_size=20, generations=30, mutation_rate=0.01, elite_size=2, selection_rate=0.3):
    mask_size = NUM_SENSORS
    log_file_path = os.path.join(BASE_DIR, f"ga_evaluation_log_{model_name}.txt")

    # 最後のジェネレーション番号を取得
    last_generation = get_last_generation(log_file_path)
    if last_generation == 0:
        population = initialize_population(population_size, mask_size)
    else:
        population = load_population(log_file_path, last_generation, population_size)

    start_time = time.time()
    best_overall_mask = None
    best_overall_score = float('-inf')

    for generation in range(last_generation, generations):
        fitness_scores = []
        for mask in population:
            adjusted_mask = adjust_mask(mask)
            avg_score = evaluate_model(model_name, adjusted_mask, generation)
            fitness_scores.append(avg_score)

            if avg_score > best_overall_score:
                best_overall_score = avg_score
                best_overall_mask = adjusted_mask

        # 適応度でソート
        sorted_population = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)

        # ログ出力部分（変更なし）
        top_3 = sorted_population[:3]
        worst = sorted_population[-2:]
        elapsed_time = time.time() - start_time
        print(f"Generation {generation + 1}, Time: {elapsed_time:.2f}s")
        for i, (mask, score) in enumerate(top_3):
            print(f"  Top {i + 1}: Score = {score:.4f}, Mask = {adjust_mask(mask)}")
        for i, (mask, score) in enumerate(worst):
            print(f"  Worst {i + 1}: Score = {score:.4f}, Mask = {adjust_mask(mask)}")

        with open(log_file_path, "a") as log_file:
            for i, (mask, score) in enumerate(sorted_population):
                log_file.write(f"Generation {generation + 1}, Rank {i + 1}, Score = {score:.4f}, Mask = {adjust_mask(mask)}\n")

        # 新しい世代の生成
        new_population = []

        # エリート選択（上位2個体）
        new_population.extend([mask for mask, _ in sorted_population[:elite_size]])

        # 選択プールの作成（上位N%の個体）
        selection_pool_size = int(population_size * selection_rate)
        selection_pool = [mask for mask, _ in sorted_population[:selection_pool_size]]
        selection_pool_scores = [score for _, score in sorted_population[:selection_pool_size]]

        # 残りの個体を生成
        while len(new_population) < population_size:
            # selection_pool(上位N%)からルーレット選択で親を選択
            parent1 = random.choices(selection_pool, weights=selection_pool_scores)[0]
            parent2 = random.choices(selection_pool, weights=selection_pool_scores)[0]

            # 交差と突然変異
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        # population_sizeに合わせる
        population = new_population[:population_size]

    return best_overall_mask

def get_last_generation(log_file_path):
    if not os.path.exists(log_file_path):
        print(f"実行ログがありません。初期状態から開始しします")
        return 0


    with open(log_file_path, 'r') as f:
        lines = f.readlines()
        if not lines:
            return 0
        last_line = lines[-1]
        generation = int(last_line.split(',')[0].split()[-1])
        print(f"実行ログが見つかりました。ここから継続実行します。世代：{generation}")
        return generation

def load_population(log_file_path, generation, population_size):
    population = []
    with open(log_file_path, 'r') as f:
        lines = f.readlines()
        for line in reversed(lines):
            if line.startswith(f"Generation {generation}"):
                mask_str = line.split('Mask = ')[-1].strip()
                mask = eval(mask_str)  # 文字列をリストに変換
                population.append(mask)
                if len(population) == population_size:
                    break
    return population

def run_genetic_algorithm():

    #世代更新
    best_mask = genetic_algorithm(MODEL_NAME, population_size=NUM_POPULATIONS, generations=NUM_GENERATIONS)

    #最終評価
    #avg_score = evaluate_model(MODEL_NAME, best_mask, NUM_GENERATIONS)
    print(f"Final Result:")
    #print(f"Model: {MODEL_NAME}")
    #print(f"Best Score: {avg_score:.4f}")
    print(f"Best Mask: {best_mask}")


#*******遺伝的アルゴリズムの実行*******
run_genetic_algorithm()
